1. Read all the TSV files into an array, write a function to extract the title and authors data
2. Go through each TSV and use title and authors to search the OCLC for OWI number
3. Go through each OWI and get:
    3.1 Earliest year published
    3.2 Holdings
    3.3 For OWI with the most holdings: VIAF -> Genders
    3.5 For each OWI where "book" in .get("format")
    Make Genders -> DDC dict, add to it manually
4. Make a dict of TITLE/AUTHOR STRING -> OWI, YEAR_PUBLISHED, GENDER(S), TOP_DDC, FAST_LIST?

In [65]:
import csv, os
from datetime import datetime
f_filen = "tabula-1963-03-17.tsv"
nf_filen = "tabula-1963-03-17-nf.tsv"
data_i = 2 #How far into the tsv file the data shows up...
fields = ['rank','title+author','rank_last_week','weeks_on_list']
stop_words_authors = ["Others"]

# Read the Data In

In [10]:
def empty(row):
    if row[0]=="":
        return True
    return False 

def extract_date(datestr):
    try:
        date = datetime.strptime(datestr[0],'%B %d, %Y')
    except ValueError:
        try:
            date = datetime.strptime(datestr[0],'%B %d,%Y')
        except ValueError:
            datestr = input("Enter the date in %B %d, %Y format: ")
            date = datetime.strptime(datestr,'%B %d, %Y')
    return date

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def header(row):
    if is_number(row[0]):
        return False
    else:
        return True
    
def read_tsv(name):
    i = 0
    arr = []
    date = ""
    with open(name) as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for row in reader:
            if (header(row)) & (not empty(row)) & (i>data_i):
                return arr
            if i ==0 :
                date = extract_date([row[1]])
            if ((i > data_i) & (not empty(row))):
                arr.append([date]+row)
            i = i+1
        return arr
    
def read_folder(folderpath):
    arr = []
    for f in os.listdir(folderpath):
        commasv = read_tsv(folderpath+f)
        arr = arr+commasv
    return arr

eighty_path = "1980/"
eighty = read_folder(path)
for x in eighty[0:100]:
    print x

[datetime.datetime(1980, 2, 3, 0, 0), '1', "SMILEY'S PEOPLE, by John le Carr\xc3\xa9. (Knopf, $10.95.) The great British spy has", '1', '7']
[datetime.datetime(1980, 2, 3, 0, 0), '2', "THE DEVIL'S ALTERNATIVE, by Frederick Forsyth. (Viking, $12.95.) Countdown", '--', '1']
[datetime.datetime(1980, 2, 3, 0, 0), '3', 'JAILBIRD, by Kurt Vonnegut. (Delacorte/Seymour Lawrence, $9.95.) From World', '2', '20']
[datetime.datetime(1980, 2, 3, 0, 0), '4', 'TRIPLE, by Ken Follett. (Arbor House, $10.95.) Power play between intelligence', '4', '22']
[datetime.datetime(1980, 2, 3, 0, 0), '5', 'MEMORIES OF ANOTHER DAY, by Harold Robbins. (Simon & Schuster, $10.95.)', '5', '12']
[datetime.datetime(1980, 2, 3, 0, 0), '6', 'THE ESTABLISHMENT, by Howard Fast. (Houghton Mifflin, $11.95.) The', '7', '17']
[datetime.datetime(1980, 2, 3, 0, 0), '7', 'PRINCESS DAISY, by Judith Krantz. (Crown, $12.95.) The fight for survival in the', '--', '1']
[datetime.datetime(1980, 2, 3, 0, 0), '8', "SOPHIE'S CHOICE, by Wil

# Get Title and Author

In [93]:
parse_corrections = {}

In [15]:
sample_title_auth = commasv[0][2]
#print sample_title_authR

#index into phrase to skip " by"
auth_i = 4

def parse_author_phrase(a_ph):
    authors = []
    end = len(a_ph)
    parens = a_ph.find("(")
    if parens != -1:
        end = parens-1
    auth_ph = a_ph[auth_i:end]
    if " and " in auth_ph:
        author_arr = auth_ph.split(" and ")
        for author in author_arr:
            if author[len(author)-1]=='.':
                author = author[:len(author)-1] # strip trailing period
            authors.append(author)
    else:
        author = auth_ph
        if author[len(author)-1]=='.':
            author = author[:len(author)-1] # strip trailing period
        if author in parse_corrections:
            author = parse_corrections[author]
        authors.append(author)
    return authors

def title_author_pair(data_str):
    split_str = data_str.split(',')
    title = ""
    auth_ph = ""
    i=0
    for phrase in split_str:    
        if i == 0:
            title = phrase
        else:
            if phrase.isupper():
                title = title + ", "+phrase
            else:
                auth_ph = auth_ph+phrase
        i = i+1
    if title in parse_corrections:
        title = parse_corrections[title]
    authors = parse_author_phrase(auth_ph)
    return (title,authors)

for x in eighty:
    ta = title_author_pair(x)
    if '$' in ta[0]:
        print ta

("SMILEY'S PEOPLE", ['John le Carr\xc3\xa9'])
("THE DEVIL'S ALTERNATIVE", ['Frederick Forsyth'])
('JAILBIRD', ['Kurt Vonnegut'])
('TRIPLE', ['Ken Follett'])
('MEMORIES OF ANOTHER DAY', ['Harold Robbins'])
('THE ESTABLISHMENT', ['Howard Fast'])
('PRINCESS DAISY', ['Judith Krantz'])
("SOPHIE'S CHOICE", ['William Styron'])
('THE TOP OF THE HILL', ['Irwin Shaw'])
('PORTRAITS', ['Cynthia Freeman'])
("THE EXECUTIONER'S SONG", ['Norman Mailler'])
('THE DEAD ZONE', ['Stephen King'])
('THE LAST ENCHANTMENT', ['Mary Stewart'])
('THE THIRD WORLD WAR: August 1985', ['General Sir John Hackett', 'Other'])
('MOVIOLA', ['Garson Kanin'])
('THE BOURNE IDENTITY', ['Robert Ludlum'])
('PRINCESS DAISY', ['Judith Krantz'])
("THE DEVIL'S ALTERNATIVE", ['Frederick Forsyth'])
('THE BLEEDING HEART', ['Marilyn French'])
("SMILEY'S PEOPLE", ['John le Carr\xc3\xa9'])
("WHO'S ON FIRST", ['William F. Buckley Jr'])
('PORTRAITS', ['Cynthia Freeman'])
('NO LOVE LOST', ['Helen Van Slyke'])
("CREEK MARY'S BLOOD", ['Dee Br

# To OCLC


In [86]:
import xml.etree.ElementTree
import json
import urllib2
import re

ta_to_oclc_dict = {} 
oclc_base = "http://classify.oclc.org/classify2/Classify?"
alph = re.compile('[^a-zA-Z]')
suffixes = ['CPA', 'CSC', 'CSJ', 'DC', 'DD', 'DDS', 'DMD', 'DO', 'DVM', 'EDD', 'ESQ', 'II', 'III', 
            'IV', 'INC', 'JD', 'JR', 'LLD', 'LTD', 'MD', 'OD', 'OSB', 'PC', 'PE', 'PHD', 'RET', 
            'RGS', 'RN', 'RNC', 'SHCJ', 'SJ', 'SNJM', 'SR', 'SSMO', 'USA', 'USAF', 'USAFR', 'USAR', 
            'USCG', 'USMC', 'USMCR', 'USN', 'USNR']

for_manual_search = []

In [49]:
def suffix(phrase):
    norm_phrase = phrase.replace(',',"").upper()
    if norm_phrase in suffixes:
        return True
    else:
        return False
    
def strip_punct(text):
    return alph.sub('', text)

def norm_text(text):
    return strip_punct(text).lower()

def url_to_data(url,f):
    req = urllib2.Request(url)
    req.add_header('User-agent', 'Mozilla 5.10')
    res = urllib2.urlopen(req)
    if(f=="xml"):
        data = xml.etree.ElementTree.parse(res).getroot()
    if(f=="json"):
        data = json.load(res)
    return data

def convert_to_search(name):
    return_string = ""
    last_ch = " "
    for ch in name:
        if (ch == " "):
            if (last_ch != "."):
                return_string = return_string + "+"
        else:
            if ch == ".":
                return_string = return_string +".+"
            else:
                return_string = return_string+ch
        last_ch = ch
    return return_string

def last_name_recursive(name,last_index):
    if last_index == 0:
        return ""
    last_phrase = name[last_index]
    if not suffix(last_phrase):
        return last_phrase
    else:
        return last_name_recursive(name, last_index-1)

def last_name(name):
    names = name.split(" ")
    last_index = len(names)-1
    last = last_name_recursive(names,last_index)
    return last

In [84]:
def find_all_works(xml):
    works = xml.find('{http://classify.oclc.org}works')
    if works is None:
        works = []
        works.append(xml.find('{http://classify.oclc.org}work'))
        if (works is None) or works == []:
            return []        
    books = []
    for w in works:
        if "book" in norm_text(w.get("format")):
            owi = w.get("owi")
            holdings = w.get("holdings")
            try:
                lyr = w.get("lyr")
            except:
                lyr = ""
            books.append({"owi":owi,"holdings":holdings,"lyr":lyr})
    return books

def new_work(w, entries):
    for e in entries:
        if w['owi'] == e["owi"]:
             return False
    return True

def get_oclc_entries(title,authors):
    url = oclc_base +"title="+convert_to_search(title)
    i = 0
    entries = []
    for a in authors:
        auth_url = url +"&author=" + last_name(a)
        xml = url_to_data(auth_url,"xml")
        try:
            works = find_all_works(xml)
        except:
            return auth_url
        if entries == []:
            entries = works
        else:
            for w in works:
                if new_work(w,entries):
                    entries = entries.append(w)
    return entries
    
get_oclc_entries('SEVEN DAYS IN MAY', ['Fletcher Knebel','Charles W. Bailey II'])    
#    print type(xml)
#    authors = extract_authors(xml) #returns a list of dicts of form {'name':'author_name','viaf':'viaf_id'}
#    print type(xml)
#    subjects = extract_subjects(xml) #returns a list of dicts of form {'subject':'subject_name','fast_id':'fast_id'}
#    ddc = extract_ddc(xml) #returns a list of dicts of form {'label':'ddc_label', 'percent':'percent_of_entries_w_label'}
#    oclc_id = extract_oid(xml)
#    holdings = extract_holdings(xml)
#    return {'authors':authors,'subjects':subjects,'ddc':ddc,'oclc_id':oclc_id,'holdings':holdings}

[{'holdings': '2275', 'lyr': '1962', 'owi': '1380647'},
 {'holdings': '82', 'lyr': '1962', 'owi': '1806398693'}]

In [92]:
for_manual_search = []

for x in eighty:
    ta = title_author_pair(x[2])
    #if ta in manual_ta:
        
    if str(ta) not in ta_to_oclc_dict:
        title = ta[0]
        author = ta[1]
        oclc = get_oclc_entries(title,author)
        if type(oclc) != str:
            ta_to_oclc_dict[str(ta)] = oclc
        else:
            manual = ta
            if manual not in for_manual_search:
                print str(ta)
                for_manual_search.append(manual)

print for_manual_search

("THE EXECUTIONER'S SONG", ['Norman Mailler'])
('COME POUR THE WINE,  $12.95.) A', ['Cynthia Freeman'])
('THE CRADLE WILL FALL,  $10.95.) A', ['Mary Higgins Clark'])
('THE SECOND COMING,  $12.95.) A', ['Walker Percy'])
('THE TENTH COMMANDMENT,  $11.95.) A', ['Lawrence Sanders'])
[("THE EXECUTIONER'S SONG", ['Norman Mailler']), ('COME POUR THE WINE,  $12.95.) A', ['Cynthia Freeman']), ('THE CRADLE WILL FALL,  $10.95.) A', ['Mary Higgins Clark']), ('THE SECOND COMING,  $12.95.) A', ['Walker Percy']), ('THE TENTH COMMANDMENT,  $11.95.) A', ['Lawrence Sanders'])]


In [97]:
for x in for_manual_search:
    if '$' in x[0]:
        dialog = "Correct this "+ t[0]
        corrected = input(dialog)
        parse_corrections[t[0]] = corrected
    else:
        for y in x[1]:
            dialog = "Correct this "+ y
            corrected = input(dialog)
            parse_corrections[y] = corrected
            

Correct this Norman MaillerNorman Mailer


SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [98]:
person = input('Enter your name: ')
print('Hello', person)

Enter your name: F


NameError: name 'F' is not defined